# Data generation
Generates data for a desired date based on the available APC data and passed through the model for load prediction.
## Generates the following files:
* `trip_plan.json`
* `vehicle_plan.json`
* `sampled_loads.pkl`

In [1]:
from tensorflow.keras import backend as K
K.clear_session()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sys
import datetime as dt
import importlib
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark import SparkConf
import numpy as np
import pickle
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate, GlobalAveragePooling1D
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
import IPython
from copy import deepcopy
from tqdm import trange, tqdm

mpl.rcParams['figure.facecolor'] = 'white'

import warnings

import pandas as pd
import swifter
pd.set_option('display.max_columns', None)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.get_logger().setLevel('INFO')
import pyspark
print(pyspark.__version__)
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '40g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .getOrCreate()

3.3.0
22/10/19 21:08:39 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/10/19 21:08:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 21:08:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 21:08:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [48]:
# Get data for the specified day
def get_apc_data_for_date(filter_date):
    print(f"Running this get_apc_data_for_date({filter_date})...")
    # filepath = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
    filepath = '/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs_20220921.parquet'
    apcdata = spark.read.load(filepath)
    apcdata.createOrReplaceTempView("apc")

    plot_date = filter_date.strftime('%Y-%m-%d')
    get_columns = ['trip_id', 'transit_date', 'arrival_time', 'scheduled_time', 'vehicle_id',
                   'block_abbr', 'stop_sequence', 'stop_name', 'stop_id_original',
                   'load', 'departure_time', 'vehicle_capacity',
                   'darksky_temperature', 
                   'darksky_humidity', 
                   'darksky_precipitation_probability', 
                   'route_direction_name', 'route_id', 'gtfs_direction_id',
                   'dayofweek',  'year', 'month', 'hour', 'zero_load_at_trip_end',
                   'sched_hdwy']
    get_str = ", ".join([c for c in get_columns])
    query = f"""
    SELECT {get_str}
    FROM apc
    WHERE (transit_date == '{plot_date}')
    ORDER BY arrival_time
    """
    apcdata = spark.sql(query)
    apcdata = apcdata.na.fill(value=0,subset=["zero_load_at_trip_end"])
    apcdata = apcdata.withColumn("route_id_dir", F.concat_ws("_", apcdata.route_id, apcdata.route_direction_name))
    apcdata = apcdata.withColumn("day", F.dayofmonth(apcdata.arrival_time))
    apcdata = apcdata.drop("route_direction_name")
    apcdata = apcdata.withColumn("load", F.when(apcdata.load < 0, 0).otherwise(apcdata.load))
    return apcdata

In [49]:
def prepare_input_data(input_df, ohe_encoder, label_encoders, num_scaler, columns, keep_columns=[], target='y_class'):
    num_columns = ['darksky_temperature', 'darksky_humidity', 'darksky_precipitation_probability', 'sched_hdwy']
    cat_columns = ['month', 'hour', 'day', 'stop_sequence', 'stop_id_original', 'year', 'time_window']
    ohe_columns = ['dayofweek', 'route_id_dir', 'is_holiday', 'is_school_break', 'zero_load_at_trip_end']

    # OHE
    input_df[ohe_encoder.get_feature_names_out()] = ohe_encoder.transform(input_df[ohe_columns]).toarray()
    # input_df = input_df.drop(columns=ohe_columns)

    # Label encoder
    for cat in cat_columns:
        encoder = label_encoders[cat]
        input_df[cat] = encoder.transform(input_df[cat])
    
    # Num scaler
    input_df[num_columns] = num_scaler.transform(input_df[num_columns])
    input_df['y_class']  = input_df.y_class.astype('int')

    if keep_columns:
        columns = keep_columns + columns
    # Rearrange columns
    input_df = input_df[columns]
    
    return input_df

def assign_data_to_bins(df, TARGET='load'):
    bins = pd.IntervalIndex.from_tuples([(-1, 6.0), (6.0, 12.0), (12.0, 55.0), (55.0, 75.0), (75.0, 100.0)])
    mycut = pd.cut(df[TARGET].tolist(), bins=bins)
    df['y_class'] = mycut.codes
    return df

In [50]:
TIMEWINDOW = 15
def add_features(df):
    df = df[df.arrival_time.notna()]
    df = df.fillna(method="bfill")
    # df = df[df.sched_hdwy.notna()]
    # df = df[df.darksky_temperature.notna()]

    df['day'] = df["arrival_time"].dt.day
    df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

    # Adding extra features
    # Holidays
    fp = os.path.join('data', 'US Holiday Dates (2004-2021).csv')
    holidays_df = pd.read_csv(fp)
    holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
    holidays_df['is_holiday'] = True
    df = df.merge(holidays_df[['Date', 'is_holiday']], left_on='transit_date', right_on='Date', how='left')
    df['is_holiday'] = df['is_holiday'].fillna(False)
    df = df.drop(columns=['Date'])
        
    # School breaks
    fp = os.path.join('data', 'School Breaks (2019-2022).pkl')
    school_break_df = pd.read_pickle(fp)
    school_break_df['is_school_break'] = True
    df = df.merge(school_break_df[['Date', 'is_school_break']], left_on='transit_date', right_on='Date', how='left')
    df['is_school_break'] = df['is_school_break'].fillna(False)
    df = df.drop(columns=['Date'])

    df['minute'] = df['arrival_time'].dt.minute
    df['minuteByWindow'] = df['minute'] // TIMEWINDOW
    df['temp'] = df['minuteByWindow'] + (df['hour'] * 60 / TIMEWINDOW)
    df['time_window'] = np.floor(df['temp']).astype('int')
    df = df.drop(columns=['minute', 'minuteByWindow', 'temp'])

    # HACK
    df = df[df['hour'] != 3]
    df = df[df['stop_sequence'] != 0]

    df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

    df = assign_data_to_bins(df, TARGET='load')
    return df

In [51]:
DATE = '2021-08-23'
date_to_predict = dt.datetime.strptime(DATE, '%Y-%m-%d')
# date_to_predict
# date_to_predict = dt.date(2021, 8, 23)
apcdata = get_apc_data_for_date(date_to_predict)
df = apcdata.toPandas()
df = add_features(df)

Running this get_apc_data_for_date(2021-08-23 00:00:00)...


In [52]:
def setup_simple_lstm_generator(num_features, num_classes, learning_rate=1e-4):
    # define model
    model = tf.keras.Sequential()
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # compile model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["sparse_categorical_accuracy"],
    )

    input_shape = (None, None, num_features)
    model.build(input_shape)
    return model

def generate_simple_lstm_predictions(input_df, model, past, future):
    past_df = input_df[0:past]
    future_df = input_df[past:]
    predictions = []
    if future == None:
        future = len(future_df)
    for f in range(future):
        pred = model.predict(past_df.to_numpy().reshape(1, *past_df.shape))
        y_pred = np.argmax(pred)
        predictions.append(y_pred)
        
        # Add information from future
        last_row = future_df.iloc[[0]]
        last_row['y_class'] = y_pred
        past_df = pd.concat([past_df[1:], last_row])
        
        # Move future to remove used row
        future_df = future_df[1:]
    return predictions

In [53]:
# Load model
latest = tf.train.latest_checkpoint('models/no_speed')
columns = joblib.load('models/LL_X_columns.joblib')
label_encoders = joblib.load('models/LL_Label_encoders.joblib')
ohe_encoder = joblib.load('models/LL_OHE_encoder.joblib')
num_scaler = joblib.load('models/LL_Num_scaler.joblib')

raw_df = deepcopy(df)
input_df = prepare_input_data(df, ohe_encoder, label_encoders, num_scaler, columns, target='y_class')

In [66]:
import random

percentiles = [(0, 6.0), (6.0, 12.0), (12.0, 55.0), (55.0, 75.0), (75.0, 100.0)]

NUM_CLASSES = 5
FUTURE = None
PAST = 5

NUM_TRIPS = None
if NUM_TRIPS == None:
    rand_trips = df.trip_id.unique().tolist()
else:
    rand_trips = random.sample(df.trip_id.unique().tolist(), NUM_TRIPS)

model = setup_simple_lstm_generator(input_df.shape[1], NUM_CLASSES)
model.load_weights(latest)

trip_res = []
load_arr = []
for trip_id in tqdm(rand_trips):
    _df = df.query("trip_id == @trip_id")
    try:
        _input_df = input_df.loc[_df.index]
        y_pred = generate_simple_lstm_predictions(_input_df, model, PAST, FUTURE)
        loads = [random.randint(percentiles[yp][0], percentiles[yp][1]) for yp in y_pred]
        
        _raw_df = raw_df.loc[_df.index]
        y_true = _raw_df[0:PAST]['load'].tolist()
        a = y_true + loads
        _raw_df['sampled_loads'] = a
        
        y_true_classes = _raw_df[0:PAST]['y_class'].tolist()
        _raw_df['y_pred_classes'] = y_true_classes + y_pred
        
        trip_res.append(_raw_df)
    except:
        print(f"FAILED:{trip_id}")
        continue

trip_res = pd.concat(trip_res)

  1%|          | 12/1364 [00:10<19:14,  1.17it/s]

100%|██████████| 1364/1364 [19:52<00:00,  1.14it/s]


In [67]:
_columns = ['trip_id', 'transit_date', 'arrival_time', 'scheduled_time', 'vehicle_id', 'vehicle_capacity', 'block_abbr', 'stop_sequence', 'stop_id_original', 'route_id_dir', 'zero_load_at_trip_end', 'y_pred_classes', 'sampled_loads']
trip_res = trip_res[_columns]

fp = 'results/sampled_loads.pkl'
trip_res.to_pickle(fp)

In [68]:
# 9300, MCSRVRF, 2021-08-23 06:37:00, 93_LOOP, 0
trip_res.query("trip_id == '243425'").head()

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,vehicle_capacity,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,y_pred_classes,sampled_loads
42229,243425,2021-08-23,2021-08-23 06:35:49,2021-08-23 06:37:00,127,45.0,9300,1,MCSRVRF,93_LOOP,0,2,34.0
42230,243425,2021-08-23,2021-08-23 06:39:00,2021-08-23 06:40:00,127,45.0,9300,4,BRO9AWF,93_LOOP,0,2,29.0
42231,243425,2021-08-23,2021-08-23 06:39:32,2021-08-23 06:41:09,127,45.0,9300,5,BRO12WN,93_LOOP,0,2,28.0
42232,243425,2021-08-23,2021-08-23 06:41:16,2021-08-23 06:43:12,127,45.0,9300,6,WES17AWN,93_LOOP,0,2,27.0
42233,243425,2021-08-23,2021-08-23 06:42:52,2021-08-23 06:44:43,127,45.0,9300,7,WES20AWN,93_LOOP,0,2,27.0


In [11]:
# 9300, MCSRVRF, 2021-08-23 06:37:00, 93_LOOP, 0
raw_df.query("trip_id == '243425'").head()

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,block_abbr,stop_sequence,stop_name,stop_id_original,load,departure_time,vehicle_capacity,darksky_temperature,darksky_humidity,darksky_precipitation_probability,route_id,gtfs_direction_id,dayofweek,year,month,hour,zero_load_at_trip_end,sched_hdwy,route_id_dir,day,is_holiday,is_school_break,time_window,y_class
42229,243425,2021-08-23,2021-08-23 06:35:49,2021-08-23 06:37:00,127,9300,1,RIVERFRONT STATION,MCSRVRF,34.0,2021-08-23 06:35:49,45.0,71.521,0.94,0.0,93,0,2,2021,8,6.0,0,600.0,93_LOOP,23,False,False,26,2
42230,243425,2021-08-23,2021-08-23 06:39:00,2021-08-23 06:40:00,127,9300,4,BROADWAY AVE & 9TH AVE S WB,BRO9AWF,29.0,2021-08-23 06:39:12,45.0,71.521,0.94,0.0,93,0,2,2021,8,6.0,0,3780.0,93_LOOP,23,False,False,26,2
42231,243425,2021-08-23,2021-08-23 06:39:32,2021-08-23 06:41:09,127,9300,5,BROADWAY AVE & 12TH AVE N WB,BRO12WN,28.0,2021-08-23 06:40:04,45.0,71.521,0.94,0.0,93,0,2,2021,8,6.0,0,3780.0,93_LOOP,23,False,False,26,2
42232,243425,2021-08-23,2021-08-23 06:41:16,2021-08-23 06:43:12,127,9300,6,WEST END AVE & 17TH AVE S WB,WES17AWN,27.0,2021-08-23 06:41:46,45.0,71.521,0.94,0.0,93,0,2,2021,8,6.0,0,838.0,93_LOOP,23,False,False,26,2
42233,243425,2021-08-23,2021-08-23 06:42:52,2021-08-23 06:44:43,127,9300,7,WEST END AVE & 20TH AVE S WB,WES20AWN,27.0,2021-08-23 06:43:08,45.0,71.521,0.94,0.0,93,0,2,2021,8,6.0,0,1800.0,93_LOOP,23,False,False,26,2


## Generate vehicle assignments here...

In [16]:
import json
# Vehicle assignments
# Each vehicle config is a dict: {vehicle_capacity, blocks}
DEFAULT_CAPACITY = 40.0
overall_vehicle_plan = {}

# TODO: run again with vehicle_capacity (above)
for vehicle_id, vehicle_df in trip_res.groupby('vehicle_id'):
    vehicle_df = vehicle_df.dropna(subset=['arrival_time']).sort_values(['scheduled_time'])
    vehicle_capacity = vehicle_df.iloc[0].vehicle_capacity
    # vehicle_capacity = DEFAULT_CAPACITY
    if np.isnan(vehicle_capacity):
        vehicle_capacity = DEFAULT_CAPACITY
    # TODO: This is not the baseline behavior
    starting_depot = 'MCC5_1'
    service_type = 'regular'
    blocks = [block for block in vehicle_df.block_abbr.unique().tolist()]
    trips = []
    for block in blocks:
        block_df = vehicle_df.query("block_abbr == @block")
        for trip in block_df.trip_id.unique().tolist():
            trips.append((str(block), str(trip)))
    overall_vehicle_plan[vehicle_id] = {'vehicle_capacity': vehicle_capacity, 'trips': trips, 'starting_depot': starting_depot, 'service_type': service_type}
    
len(overall_vehicle_plan)

# Number of overload buses
#   "42": {
#     "service_type": "overload",
#     "starting_depot": "MCC5_1",
#     "trips": [
#     ],
#     "vehicle_capacity": 55.0
#   }
OVERLOAD_BUSES = 2
for vehicle_id in range(41, 41 + OVERLOAD_BUSES):
    overall_vehicle_plan[str(vehicle_id)] = {'vehicle_capacity': 55.0, 'trips': [], "starting_depot": "MCC5_1", 'service_type': "overload"}
    
with open('results/vehicle_plan.json', 'w') as fp:
    json.dump(overall_vehicle_plan, fp, sort_keys=True, indent=2)

## Setting up Trip plan (sanity check)

In [28]:
trip_res

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,sampled_loads
0,245109,2021-08-23,2021-08-23 05:12:36,2021-08-23 05:20:00,733,300,1,MCC5_5,3_FROM DOWNTOWN,0,6.0
1,245109,2021-08-23,2021-08-23 05:22:16,2021-08-23 05:22:36,733,300,2,6AVDEASN,3_FROM DOWNTOWN,0,6.0
2,245109,2021-08-23,2021-08-23 05:22:50,2021-08-23 05:23:17,733,300,3,6AVCHUSN,3_FROM DOWNTOWN,0,6.0
3,245109,2021-08-23,2021-08-23 05:24:18,2021-08-23 05:24:48,733,300,4,8ABROSN,3_FROM DOWNTOWN,0,8.0
4,245109,2021-08-23,2021-08-23 05:25:52,2021-08-23 05:25:15,733,300,5,BRO9AWF,3_FROM DOWNTOWN,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
42348,246345,2021-08-23,2021-08-23 23:38:02,2021-08-23 23:39:18,1826,9975,20,ANNPREEF,50_FROM DOWNTOWN,0,5.0
42349,246345,2021-08-23,2021-08-23 23:38:36,2021-08-23 23:40:00,1826,9975,21,CXOANNEX,50_FROM DOWNTOWN,0,4.0
42350,246345,2021-08-23,2021-08-23 23:40:58,2021-08-23 23:41:45,1826,9975,22,CXONVLWS,50_FROM DOWNTOWN,0,0.0
42351,246345,2021-08-23,2021-08-23 23:42:34,2021-08-23 23:44:39,1826,9975,23,CXODVDSN,50_FROM DOWNTOWN,0,2.0


In [32]:
# Create a dict of {[block: {trip_ids:[]}, 'block'....]}
# trip_id dict = {'route_id', route_direction_name', 'stop_id':[], 'schedule_time':[]}
# Use block as grouper in baseline
overall_block_plan = {}
for block_abbr, block_df in trip_res.groupby('block_abbr'):
    block_df = block_df.dropna(subset=['arrival_time']).sort_values(['scheduled_time'])
    trip_ids = block_df.trip_id.unique().tolist()
    start_time = block_df[block_df['trip_id'] == trip_ids[0]].iloc[0]['scheduled_time'].strftime('%Y-%m-%d %H:%M:%S')
    end_time = block_df[block_df['trip_id'] == trip_ids[-1]].iloc[-1]['scheduled_time'].strftime('%Y-%m-%d %H:%M:%S')
    overall_block_plan[block_abbr] = {'trip_ids': trip_ids,
                                      'start_time': start_time,
                                      'end_time': end_time}

overall_trip_plan = {}
for trip_id, trip_df in trip_res.groupby('trip_id'):
    trip_df = trip_df.dropna(subset=['arrival_time']).sort_values(['scheduled_time'])
    route_id_dir = trip_df.iloc[0].route_id_dir
    route_id = int(route_id_dir.split("_")[0])
    route_direction = route_id_dir.split("_")[1]
    zero_load_at_trip_end = trip_df.iloc[-1].zero_load_at_trip_end.tolist()
    scheduled_time = trip_df.scheduled_time.dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
    stop_sequence = trip_df.stop_sequence.tolist()
    stop_sequence = list(range(0, len(stop_sequence)))
    # stop_sequence = [ss - 1 for ss in stop_sequence]
    stop_id_original = trip_df.stop_id_original.tolist()
    
    overall_trip_plan[trip_id] = {'route_id': route_id, 
                                  'route_direction': route_direction, 
                                  'scheduled_time': scheduled_time, 
                                  'stop_sequence': stop_sequence, 
                                  'stop_id_original': stop_id_original,
                                  'zero_load_at_trip_end':zero_load_at_trip_end,
                                  'last_stop_sequence': stop_sequence[-1],
                                  'last_stop_id': stop_id_original[-1]}

len(overall_trip_plan), len(overall_block_plan)

with open('results/trip_plan.json', 'w') as fp:
    json.dump(overall_trip_plan, fp, sort_keys=True, indent=2)

In [23]:
display(trip_res.query("trip_id == '246303'"))
print(trip_res.query("trip_id == '246303'").shape)

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,sampled_loads
42202,246303,2021-08-23,2021-08-23 05:58:33,2021-08-23 06:13:00,1204,8601,1,SMRY,86_TO NASHVILLE,0,6.0
42203,246303,2021-08-23,2021-08-23 06:16:52,2021-08-23 06:16:15,1204,8601,2,MURFERWF,86_TO NASHVILLE,0,6.0
42204,246303,2021-08-23,2021-08-23 06:18:18,2021-08-23 06:18:14,1204,8601,3,MURVETWN,86_TO NASHVILLE,0,6.0
42205,246303,2021-08-23,2021-08-23 06:19:00,2021-08-23 06:19:00,1204,8601,4,MURREBWF,86_TO NASHVILLE,0,6.0
42206,246303,2021-08-23,2021-08-23 06:19:24,2021-08-23 06:20:05,1204,8601,5,MURSTOWF,86_TO NASHVILLE,0,7.0
42207,246303,2021-08-23,2021-08-23 06:20:26,2021-08-23 06:20:38,1204,8601,6,MURJACWF,86_TO NASHVILLE,0,0.0
42208,246303,2021-08-23,2021-08-23 06:21:16,2021-08-23 06:21:45,1204,8601,7,MURCITWN,86_TO NASHVILLE,0,6.0
42209,246303,2021-08-23,2021-08-23 06:47:58,2021-08-23 07:20:35,1204,8601,8,HERFAIWF,86_TO NASHVILLE,0,6.0
42210,246303,2021-08-23,2021-08-23 06:48:06,2021-08-23 07:20:58,1204,8601,9,HERDRIWN,86_TO NASHVILLE,0,4.0
42211,246303,2021-08-23,2021-08-23 06:48:40,2021-08-23 07:22:04,1204,8601,10,HERLINWF,86_TO NASHVILLE,0,5.0


(13, 11)


In [ ]:
raw_df.query("trip_id == '243423'").shape

In [34]:
route_id_dir = 
trip_res.query("route_id_dir == @route_id_dir and block_abbr == @block and stop_id_original == @stop_id_original[@i] and scheduled_time == @scheduled_time[@i]").iloc[0]['sampled_loads']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42353 entries, 0 to 42352
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trip_id                42353 non-null  object        
 1   transit_date           42353 non-null  datetime64[ns]
 2   arrival_time           42353 non-null  datetime64[ns]
 3   scheduled_time         42353 non-null  datetime64[ns]
 4   vehicle_id             42353 non-null  object        
 5   block_abbr             42353 non-null  int32         
 6   stop_sequence          42353 non-null  int32         
 7   stop_id_original       42353 non-null  object        
 8   route_id_dir           42353 non-null  object        
 9   zero_load_at_trip_end  42353 non-null  int32         
 10  sampled_loads          42353 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int32(3), object(4)
memory usage: 4.4+ MB


## Getting ons and offs from sampled loads

In [40]:
import pandas as pd

def compute_ons_offs(s):
    curr_load = s['sampled_loads']
    next_load = s['next_load']
    if next_load > curr_load:
        ons = next_load - curr_load
        offs = 0
    elif next_load < curr_load:
        ons = 0
        offs = curr_load - next_load
    else:
        ons = 0
        offs = 0
        
    return ons, offs
    
fp = 'results/sampled_loads.pkl'
trip_res = pd.read_pickle(fp)
sampled_ons_offs = []
for trip_id, trip_id_df in tqdm(trip_res.groupby(['transit_date', 'trip_id'])):
    tdf = trip_id_df.sort_values('stop_sequence').reset_index(drop=True)
    tdf['ons'] = 0
    tdf['offs'] = 0
    tdf['next_load'] = tdf['sampled_loads'].shift(-1)
    
    # Intermediate stops
    tdf[['ons', 'offs']] = tdf.apply(compute_ons_offs, axis=1, result_type="expand")
    
    # first and last stops
    tdf.at[0, 'ons'] = tdf.iloc[0]['sampled_loads']
    tdf.at[len(tdf) - 1, 'offs'] = tdf.iloc[-1]['sampled_loads']
    sampled_ons_offs.append(tdf)
    
sampled_ons_offs = pd.concat(sampled_ons_offs)
sampled_ons_offs = sampled_ons_offs.drop('next_load', axis=1)

fp = 'results/sampled_ons_offs.pkl'
sampled_ons_offs.to_pickle(fp)

100%|██████████| 1364/1364 [00:04<00:00, 289.18it/s]


In [43]:
sampled_ons_offs.query("trip_id == '246346'")

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,vehicle_capacity,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,sampled_loads,ons,offs
0,246346,2021-08-23,2021-08-23 22:16:00,2021-08-23 22:31:00,1826,40.0,9975,1,WALMARTC,50_TO DOWNTOWN,0,0.0,0.0,0.0
1,246346,2021-08-23,2021-08-23 22:34:24,2021-08-23 22:32:52,1826,40.0,9975,2,CXIDVDSN,50_TO DOWNTOWN,0,0.0,0.0,0.0
2,246346,2021-08-23,2021-08-23 22:36:20,2021-08-23 22:36:10,1826,40.0,9975,3,CXINVLWS,50_TO DOWNTOWN,0,0.0,4.0,0.0
3,246346,2021-08-23,2021-08-23 22:37:12,2021-08-23 22:38:00,1826,40.0,9975,4,CXIANNEX,50_TO DOWNTOWN,0,4.0,0.0,0.0
4,246346,2021-08-23,2021-08-23 22:39:12,2021-08-23 22:38:51,1826,40.0,9975,5,ANNCHAWM,50_TO DOWNTOWN,0,4.0,0.0,3.0
5,246346,2021-08-23,2021-08-23 22:40:20,2021-08-23 22:40:18,1826,40.0,9975,6,PREWILNN,50_TO DOWNTOWN,0,1.0,5.0,0.0
6,246346,2021-08-23,2021-08-23 22:42:30,2021-08-23 22:42:46,1826,40.0,9975,7,CXIAMERI,50_TO DOWNTOWN,0,6.0,0.0,3.0
7,246346,2021-08-23,2021-08-23 22:43:36,2021-08-23 22:44:08,1826,40.0,9975,8,CHABALEF,50_TO DOWNTOWN,0,3.0,0.0,2.0
8,246346,2021-08-23,2021-08-23 22:43:56,2021-08-23 22:44:50,1826,40.0,9975,9,CHALELEM,50_TO DOWNTOWN,0,1.0,5.0,0.0
9,246346,2021-08-23,2021-08-23 22:46:13,2021-08-23 22:46:00,1826,40.0,9975,10,CXIWTBRG,50_TO DOWNTOWN,0,6.0,0.0,0.0


In [46]:
sampled_ons_offs.vehicle_id.unique().shape

(100,)

## Generating new chains

In [44]:
import random
import pandas as pd

def compute_ons_offs(s):
    curr_load = s['sampled_loads']
    next_load = s['next_load']
    if next_load > curr_load:
        ons = next_load - curr_load
        offs = 0
    elif next_load < curr_load:
        ons = 0
        offs = curr_load - next_load
    else:
        ons = 0
        offs = 0
        
    return ons, offs

percentiles = [(0, 6.0), (6.0, 12.0), (12.0, 55.0), (55.0, 75.0), (75.0, 100.0)]
fp = 'results/sampled_loads.pkl'
trip_res = pd.read_pickle(fp)
loads = [random.randint(percentiles[yp][0], percentiles[yp][1]) for yp in trip_res.y_pred_classes]
trip_res['sampled_loads'] = loads

sampled_ons_offs = []
for trip_id, trip_id_df in tqdm(trip_res.groupby(['transit_date', 'trip_id'])):
    tdf = trip_id_df.sort_values('stop_sequence').reset_index(drop=True)
    tdf['stop_sequence'] = list(range(1, len(tdf) + 1))
    tdf['ons'] = 0
    tdf['offs'] = 0
    tdf['next_load'] = tdf['sampled_loads'].shift(-1)
    
    # Intermediate stops
    tdf[['ons', 'offs']] = tdf.apply(compute_ons_offs, axis=1, result_type="expand")
    
    # first and last stops
    tdf.at[0, 'ons'] = tdf.iloc[0]['sampled_loads']
    tdf.at[len(tdf) - 1, 'offs'] = tdf.iloc[-1]['sampled_loads']
    sampled_ons_offs.append(tdf)
    
df = pd.concat(sampled_ons_offs)
df = df.drop('next_load', axis=1)

display(df)
df['key_pair'] = list(zip(df.route_id_dir, 
                          df.block_abbr,
                          df.stop_sequence,
                          df.stop_id_original, 
                          df.scheduled_time))
df = df.set_index('key_pair')
sampled_ons_offs_dict = df.drop(['trip_id', 'route_id_dir', 'block_abbr', 'stop_id_original', 
                                 'scheduled_time', 'transit_date', 'arrival_time', 'vehicle_capacity',
                                 'vehicle_id', 'zero_load_at_trip_end', 'y_pred_classes',
                                 'stop_sequence'], axis=1).to_dict('index')
sampled_ons_offs_dict

import pickle 

with open('results/sampled_ons_offs_dict.pkl', 'wb') as handle:
    pickle.dump(sampled_ons_offs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 1364/1364 [00:05<00:00, 244.60it/s]


,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,vehicle_capacity,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,y_pred_classes,sampled_loads,ons,offs
0,241616,2021-08-23,2021-08-23 14:06:24,2021-08-23 14:15:00,1701,40.0,1400,1,MCC4_20,14_FROM DOWNTOWN,0,1,9,9.0,0.0
1,241616,2021-08-23,2021-08-23 14:18:00,2021-08-23 14:16:28,1701,40.0,1400,2,UNI2AEF,14_FROM DOWNTOWN,0,2,16,33.0,0.0
2,241616,2021-08-23,2021-08-23 14:19:12,2021-08-23 14:18:12,1701,40.0,1400,3,1SWOONM,14_FROM DOWNTOWN,0,2,49,0.0,23.0
3,241616,2021-08-23,2021-08-23 14:19:40,2021-08-23 14:19:07,1701,40.0,1400,4,1SJAMNM,14_FROM DOWNTOWN,0,2,26,9.0,0.0
4,241616,2021-08-23,2021-08-23 14:19:50,2021-08-23 14:19:29,1701,40.0,1400,5,1SOLDNN,14_FROM DOWNTOWN,0,2,35,0.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,246346,2021-08-23,2021-08-23 22:55:38,2021-08-23 22:58:15,1826,40.0,9975,20,CHA20AEF,50_TO DOWNTOWN,0,0,6,0.0,4.0
20,246346,2021-08-23,2021-08-23 22:58:04,2021-08-23 22:59:43,1826,40.0,9975,21,CXI17AVE,50_TO DOWNTOWN,0,0,2,3.0,0.0
21,246346,2021-08-23,2021-08-23 23:01:30,2021-08-23 23:02:16,1826,40.0,9975,22,CXINGULC,50_TO DOWNTOWN,0,0,5,0.0,5.0
22,246346,2021-08-23,2021-08-23 23:03:30,2021-08-23 23:03:59,1826,40.0,9975,23,CHA7AEN,50_TO DOWNTOWN,0,0,0,0.0,0.0


In [45]:
# df[('23_FROM DOWNTOWN', 2310, 'DWMRT', pd.Timestamp('2021-08-23 05:41:00'))]
df.query("route_id_dir == '23_FROM DOWNTOWN' and block_abbr == 2310 and stop_id_original == 'DWMRT' and scheduled_time == '2021-08-23 05:41:00'")

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,vehicle_capacity,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,y_pred_classes,sampled_loads,ons,offs
key_pair,,,,,,,,,,,,,,,
"(23_FROM DOWNTOWN, 2310, 32, DWMRT, 2021-08-23 05:41:00)",245847,2021-08-23,2021-08-23 05:40:10,2021-08-23 05:41:00,1904,40.0,2310,32,DWMRT,23_FROM DOWNTOWN,0,0,6,0.0,1.0
"(23_FROM DOWNTOWN, 2310, 33, DWMRT, 2021-08-23 05:41:00)",245847,2021-08-23,2021-08-23 05:41:15,2021-08-23 05:41:00,1904,40.0,2310,33,DWMRT,23_FROM DOWNTOWN,0,0,5,0.0,3.0


In [46]:
sampled_ons_offs_dict[('7_TO DOWNTOWN', 5692, 20, 'MCC5_9', pd.Timestamp('2021-08-23 14:39:00'))]

{'sampled_loads': 5, 'ons': 0.0, 'offs': 5.0}

In [47]:
# df.query("route_id_dir == '7_TO DOWNTOWN' and block_abbr == 5692 and stop_sequence == 20 and stop_id_original == 'MCC5_9'")
df.query("route_id_dir == '7_TO DOWNTOWN' and block_abbr == 5692")

,trip_id,transit_date,arrival_time,scheduled_time,vehicle_id,vehicle_capacity,block_abbr,stop_sequence,stop_id_original,route_id_dir,zero_load_at_trip_end,y_pred_classes,sampled_loads,ons,offs
key_pair,,,,,,,,,,,,,,,
"(7_TO DOWNTOWN, 5692, 1, HBHS, 2021-08-23 14:10:00)",246343,2021-08-23,2021-08-23 14:05:08,2021-08-23 14:10:00,128,45.0,5692,1,HBHS,7_TO DOWNTOWN,0,2,18,18.0,0.0
"(7_TO DOWNTOWN, 5692, 2, HILGRANN, 2021-08-23 14:11:01)",246343,2021-08-23,2021-08-23 14:12:42,2021-08-23 14:11:01,128,45.0,5692,2,HILGRANN,7_TO DOWNTOWN,0,2,48,0.0,35.0
"(7_TO DOWNTOWN, 5692, 3, HILWOONM, 2021-08-23 14:12:06)",246343,2021-08-23,2021-08-23 14:14:36,2021-08-23 14:12:06,128,45.0,5692,3,HILWOONM,7_TO DOWNTOWN,0,2,13,2.0,0.0
"(7_TO DOWNTOWN, 5692, 4, HILGOLNF, 2021-08-23 14:13:09)",246343,2021-08-23,2021-08-23 14:15:08,2021-08-23 14:13:09,128,45.0,5692,4,HILGOLNF,7_TO DOWNTOWN,0,2,15,0.0,0.0
"(7_TO DOWNTOWN, 5692, 5, HILLOMNF, 2021-08-23 14:14:11)",246343,2021-08-23,2021-08-23 14:15:58,2021-08-23 14:14:11,128,45.0,5692,5,HILLOMNF,7_TO DOWNTOWN,0,2,15,0.0,9.0
"(7_TO DOWNTOWN, 5692, 6, 21AWOONF, 2021-08-23 14:15:43)",246343,2021-08-23,2021-08-23 14:16:48,2021-08-23 14:15:43,128,45.0,5692,6,21AWOONF,7_TO DOWNTOWN,0,0,6,0.0,0.0
"(7_TO DOWNTOWN, 5692, 7, 21ABLANN, 2021-08-23 14:17:23)",246343,2021-08-23,2021-08-23 14:17:46,2021-08-23 14:17:23,128,45.0,5692,7,21ABLANN,7_TO DOWNTOWN,0,0,6,0.0,2.0
"(7_TO DOWNTOWN, 5692, 8, 21AFAINN, 2021-08-23 14:18:38)",246343,2021-08-23,2021-08-23 14:18:30,2021-08-23 14:18:38,128,45.0,5692,8,21AFAINN,7_TO DOWNTOWN,0,0,4,0.0,0.0
"(7_TO DOWNTOWN, 5692, 9, 21AACKNN, 2021-08-23 14:19:04)",246343,2021-08-23,2021-08-23 14:19:00,2021-08-23 14:19:04,128,45.0,5692,9,21AACKNN,7_TO DOWNTOWN,0,0,4,0.0,4.0
